In [19]:
import requests
from asf_hyp3 import API
from asf_hyp3 import scripts
from asf_hyp3.scripts import download_products

In [20]:
# Hardcoded values

# polarization
polarization = "VV%2bVH" # VV+VH

# platform # question: do we want images from both Sentinel 1 satellites, or just one?
platform = "SB" # both platforms for this one 

# processingLevel. only want GRD-HD images
processing_level = "GRD_HD"

# output format
output = "JSON"

In [21]:
# Read metadata from file, other metadata is hardcoded in file
metadata_dict = {
    "coordinate_list": [],
    "flightDirection": None,
    "start": None,
    "end": None,
    "API_username": None
}
with open("Coordinate_Metadata-SB.txt", "r") as metadata: # metadata.txt
    for line in metadata:
        line = line.strip()
        if len(line) == 0:
            continue
            
        key, val = line.split(": ")
        
        if key == "Coordinate":
            metadata_dict["coordinate_list"].append(val.strip())
            continue
            
        metadata_dict[key.strip()] = val.strip()

In [22]:
metadata_dict

{'coordinate_list': ['-75.9457501 -8.7161387',
  '-75.7961798 -9.2015495',
  '-75.6708379 -9.3966445',
  '-75.2827226 -9.309832',
  '-75.2375801 -9.1067715',
  '-75.1433027 -8.9721934',
  '-74.9407121 -8.8578926',
  '-74.8466835 -8.6612354',
  '-74.6178159 -8.5487194',
  '-74.5253512 -8.7493162',
  '-74.6089202 -8.8202284',
  '-74.6780937 -9.0940391',
  '-74.5894394 -9.3031764',
  '-74.6743086 -9.3944726',
  '-74.4907272 -9.6187806',
  '-74.4006082 -10.070519',
  '-74.2316607 -10.1850516',
  '-74.2767111 -10.3740743',
  '-74.1326736 -10.6049109',
  '-74.2298516 -10.6354954',
  '-74.3856808 -10.8117675',
  '-74.4849223 -10.8364565',
  '-74.5000809 -10.9959983',
  '-74.3736546 -11.0565222',
  '-74.0917783 -10.9868144',
  '-73.9782716 -11.1206268',
  '-73.8613035 -11.14553',
  '-73.8541103 -10.9549058',
  '-73.7572 -10.7359044',
  '-73.6103961 -10.8465736',
  '-73.4826031 -11.065242',
  '-73.4497583 -11.368083',
  '-73.3313307 -11.3112647',
  '-73.026509 -11.2640846',
  '-72.8344461 -11.3

In [23]:
base_url_list = ["https://api.daac.asf.alaska.edu/services/search/param?"]

In [24]:
# add polarization parameter
base_url_list.append("polarization=" + polarization + "&")

# add flight direction parameter
base_url_list.append("flightDirection=" + metadata_dict["flightDirection"] + "&")

# add platform
base_url_list.append("platform=" + platform + "&")

In [25]:
# add polygon to url

polygon = []
# trying to make this: intersectsWith=polygon((-119.543 37.925, -118.443 37.7421, -118.682 36.8525, -119.77 37.0352, -119.543 37.925 ))
# using URL encoding 
polygon.append("polygon%28%28")
for coord in metadata_dict["coordinate_list"]:
    coord = coord.replace(" ", "+")
    polygon.append(coord)
    polygon.append(",")
    
# remove the last ", "
polygon.pop()
# add closing parentheses 
polygon.append("%29%29")
# convert list of strings to string
polygon_str = "".join(polygon)
# append parameter to url
base_url_list.append("intersectsWith=" + polygon_str + "&")

In [26]:
# add dates to url
base_url_list.append("start=" + metadata_dict["start"] + "&" + "end=" + metadata_dict["end"] + "&")

In [27]:
# add processing level to url
base_url_list.append("processingLevel=" + processing_level + "&")

In [28]:
# add output type
base_url_list.append("output=" + output)

In [29]:
# convert list of strings to string
base_url = "".join(base_url_list)

In [30]:
base_url

'https://api.daac.asf.alaska.edu/services/search/param?polarization=VV%2bVH&flightDirection=DESCENDING&platform=SB&intersectsWith=polygon%28%28-75.9457501+-8.7161387,-75.7961798+-9.2015495,-75.6708379+-9.3966445,-75.2827226+-9.309832,-75.2375801+-9.1067715,-75.1433027+-8.9721934,-74.9407121+-8.8578926,-74.8466835+-8.6612354,-74.6178159+-8.5487194,-74.5253512+-8.7493162,-74.6089202+-8.8202284,-74.6780937+-9.0940391,-74.5894394+-9.3031764,-74.6743086+-9.3944726,-74.4907272+-9.6187806,-74.4006082+-10.070519,-74.2316607+-10.1850516,-74.2767111+-10.3740743,-74.1326736+-10.6049109,-74.2298516+-10.6354954,-74.3856808+-10.8117675,-74.4849223+-10.8364565,-74.5000809+-10.9959983,-74.3736546+-11.0565222,-74.0917783+-10.9868144,-73.9782716+-11.1206268,-73.8613035+-11.14553,-73.8541103+-10.9549058,-73.7572+-10.7359044,-73.6103961+-10.8465736,-73.4826031+-11.065242,-73.4497583+-11.368083,-73.3313307+-11.3112647,-73.026509+-11.2640846,-72.8344461+-11.386152,-72.5258822+-11.4393599,-72.2176641+-11.300

In [31]:
r = requests.get(base_url)

In [32]:
r.json()

[[{'absoluteOrbit': '23326',
   'beamMode': 'IW',
   'beamModeType': 'IW',
   'beamSwath': None,
   'browse': 'https://datapool.asf.alaska.edu/BROWSE/SB/S1B_IW_GRDH_1SDV_20200911T102222_20200911T102247_023326_02C4CB_3A40.jpg',
   'catSceneId': None,
   'centerLat': '-11.8794',
   'centerLon': '-71.8392',
   'collectionName': None,
   'configurationName': 'Interferometric Wide. 250 km swath, 5 m x 20 m spatial resolution and burst synchronization for interferometry. IW is considered to be the standard mode over land masses.',
   'doppler': '0',
   'downloadUrl': 'https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_IW_GRDH_1SDV_20200911T102222_20200911T102247_023326_02C4CB_3A40.zip',
   'farEndLat': '-12.36408',
   'farEndLon': '-73.17054',
   'farStartLat': '-10.864187',
   'farStartLon': '-72.802917',
   'faradayRotation': None,
   'fileName': 'S1B_IW_GRDH_1SDV_20200911T102222_20200911T102247_023326_02C4CB_3A40.zip',
   'finalFrame': '3823',
   'firstFrame': '3823',
   'flightDirection': 'DE

In [33]:
json_output = r.json()[0]

In [34]:
# get a list of granule names/ids to request processing in Hyp3
scene_names = [scene['granuleName'] for scene in json_output]

In [35]:
paths = set()
frames = set()
combos = set()

for scene in json_output:
    path = int(scene['relativeOrbit'])
    frame = int(scene['frameNumber'])
    
    paths.add(path)
    frames.add(frame)
    combos.add((path, frame))

In [36]:
print(sorted(list(paths)))
print(sorted(list(frames)))
print(sorted(list(combos)))

[25, 98, 171]
[616, 617, 621, 622, 626, 627, 628, 631, 632]
[(25, 621), (25, 622), (25, 626), (25, 627), (25, 628), (25, 631), (25, 632), (98, 617), (98, 622), (98, 627), (171, 616), (171, 617), (171, 621), (171, 622), (171, 626)]


In [37]:
scene_names

['S1B_IW_GRDH_1SDV_20200911T102222_20200911T102247_023326_02C4CB_3A40',
 'S1B_IW_GRDH_1SDV_20200911T102157_20200911T102222_023326_02C4CB_4EF7',
 'S1B_IW_GRDH_1SDV_20200911T102132_20200911T102157_023326_02C4CB_FA2F',
 'S1B_IW_GRDH_1SDV_20200909T103806_20200909T103831_023297_02C3EF_B720',
 'S1B_IW_GRDH_1SDV_20200909T103741_20200909T103806_023297_02C3EF_F658',
 'S1B_IW_GRDH_1SDV_20200904T103017_20200904T103042_023224_02C1AB_22FE',
 'S1B_IW_GRDH_1SDV_20200904T102952_20200904T103017_023224_02C1AB_2B64',
 'S1B_IW_GRDH_1SDV_20200904T102927_20200904T102952_023224_02C1AB_7CF1',
 'S1B_IW_GRDH_1SDV_20200830T102222_20200830T102247_023151_02BF51_96BE',
 'S1B_IW_GRDH_1SDV_20200830T102157_20200830T102222_023151_02BF51_C508',
 'S1B_IW_GRDH_1SDV_20200830T102132_20200830T102157_023151_02BF51_7CC3',
 'S1B_IW_GRDH_1SDV_20200828T103806_20200828T103831_023122_02BE6F_0001',
 'S1B_IW_GRDH_1SDV_20200828T103741_20200828T103806_023122_02BE6F_DB79',
 'S1B_IW_GRDH_1SDV_20200823T103017_20200823T103042_023049_02BC3B

In [38]:
len(scene_names)

860

In [ ]:
api = API('lemarpopal')

In [ ]:
api.login()

In [ ]:
api.one_time_process('S1A_IW_GRDH_1SDV_20150628T102116_20150628T102146_006572_008BFF_2503', 99)

In [ ]:
api.get_subscriptions()

In [ ]:
download_products.download_products(api)